In [1]:
from langchain_community.document_loaders import PyPDFLoader, CSVLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings, ChatOllama
from langchain_chroma import Chroma
from langchain_core.prompts import PromptTemplate
from typing_extensions import List, TypedDict
from langchain_core.documents import Document
from langgraph.graph import START, StateGraph
import getpass
import os

In [2]:
cards_loader = CSVLoader("./data/cards.csv")
doc1_loader = PyPDFLoader("./data/doc1.pdf")
doc2_loader = PyPDFLoader("./data/doc2.pdf")

In [3]:
cards = cards_loader.load()
doc1 = doc1_loader.load()
doc2 = doc2_loader.load()

In [4]:
csv_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200, chunk_overlap=0, add_start_index=True
)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=700, chunk_overlap=70, add_start_index=True
)

In [5]:
card_splits = csv_splitter.split_documents(cards)
doc1_splits = text_splitter.split_documents(doc1)
doc2_splits = text_splitter.split_documents(doc2)

In [6]:
embeddings = OllamaEmbeddings(model="bge-m3")
vector_store = Chroma(embedding_function=embeddings)

In [7]:
_ = vector_store.add_documents(documents=card_splits)
print("Cards done")
_ = vector_store.add_documents(documents=doc1_splits)
print("doc1 done")
_ = vector_store.add_documents(documents=doc2_splits)
print("doc2 done")

Cards done
doc1 done
doc2 done


In [13]:
prompt = PromptTemplate.from_template(
    """
    Ты помощник-ассистент, который отвечает на вопросы клиента.
    Используй следующий контекст для информации и ответь на вопрос клиента.
    Если ты не знаешь или не нашел ответа, так и скажи.
    Если ты нашел больше одной информации, то подробно скажи о каждой из них,
    но пропусти похожую но не связанную информацию.
    Вопрос: {question} 
    Контекст: {context} 
    Ответ:
    """
)

In [14]:
llm = ChatOllama(model="llama3.1:8b")

In [15]:
# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    for doc in retrieved_docs:
        print(f"{doc.metadata}\n")
        print(f"{doc.page_content}\n")
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [19]:
result = graph.invoke({"question": "Сколько стоит смс оповещение"})

{'row': 14, 'source': './data/cards.csv', 'start_index': 0}

Service: Плата за услугу «Оповещение об операциях»
Condition: с абонентским номером Тинькофф Мобайл
Tariff: 59 руб. в месяц

{'row': 15, 'source': './data/cards.csv', 'start_index': 0}

Service: Плата за услугу «Оповещение об операциях»
Condition: в прочих случаях
Tariff: 99 руб. в месяц

{'page': 13, 'page_label': '14', 'source': './data/doc1.pdf', 'start_index': 0}

14 из 72
4.14. Информирование, Оповещение об операциях и SMS-запросы.
4.14.1. Информирование — сервис Банка, в рамках которого на Абонентский номер и/или Абонентское устройство Клиента 
направляются Сообщения:
• об изменении статуса Договора, Заявки;
• об осуществлении нефинансовых действий по Договору;
• об изменении статуса Карты;
• иные сведения по усмотрению Банка.
4.14.2. Оповещение об операциях — сервис Банка, подключаемый по желанию Клиента и позволяющий Клиенту получать 
в режиме реального времени на Абонентский номер и/или Абонентское устройство Сообщен

In [20]:
print(f'Ответ: {result["answer"]}')

Ответ: Вопрос клиента: Сколько стоит смс оповещение 

На основе контекста, я нашел следующую информацию:

- Для абонентского номера Тинькофф Мобайл и тарифа 59 руб. в месяц, плата за услугу "Оповещение об операциях" не указана.
- Для прочих случаев (не связанных с абонентским номером Тинькофф Мобайл) и тарифа 99 руб. в месяц, плата за услугу "Оповещение об операциях" составляет 99 руб. в месяц.

Однако, я не нашел информации о стоимости СМС оповещения. Похожая информация есть в пункте 4.14.1 "Информирование", где упоминаются Сообщения направляемые на Абонентский номер, но эта информация не связана с оповещением об операциях.

Кроме того, я нашел информацию о бесплатном пополнении для партнеров Тинькофф до 150 000 руб. за расчетный период, но она не имеет отношения к вопросу клиента.

В заключении, для абонентского номера Тинькофф Мобайл и тарифа 59 руб. в месяц, я не нашел информации о стоимости СМС оповещения, а для прочих случаев (не связанных с абонентским номером Тинькофф Мобайл) и